## Языковое моделирование

Языковые модели - важнейшая часть современного NLP. Практически во всех задачах, связанных с обработкой текста, напрямую или косвенно используются языковые модели. А наиболее известные недавние прорывы в области - это по большей части новые подходы к языковому моделированию. ELMO, BERT, GPT - это языковые модели.

Это достаточно сложная тема, поэтому будем разбирать постепенно. Сегодня разберём самые основы. Научимся приписывать вероятность последовательности слов и попробуем генерировать текст.

In [48]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 

Возьмем два разных корпуса: новостной и сообщения с 2ch.

In [5]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('2ch_corpus.txt').read()
news = open('lenta.txt').read()

По длине они сопоставимы.

In [6]:
print("Длина 1 -", len(dvach))
print("Длина 2 -", len(news))

Длина 1 - 11638405
Длина 2 - 11536552


Напишем простую функцию для нормализации. Удалять пунктуацию и приводить к нижнему регистру, строго говоря не стоит, сгенерированный текст так будет не похож на настоящий. Но это немного упростит нам работу. 

In [7]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Сравним тексты по токенам

In [8]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [9]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 1858941
Длина корпуса новостных текстов в токенах -  1505789


И по уникальным токенам

In [10]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 189515
Уникальный токенов в корпусе новостных текстов -  116302


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [11]:
from collections import Counter

In [12]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)


In [13]:
vocab_dvach.most_common(10)

[('и', 55892),
 ('в', 48853),
 ('не', 46602),
 ('на', 29660),
 ('что', 26668),
 ('я', 21734),
 ('а', 21310),
 ('с', 21080),
 ('это', 17727),
 ('ты', 15469)]

In [14]:
vocab_news.most_common(10)

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

Для того, чтобы превратить абсолютные частоты в вероятности, разделим на общее число слов в каждом корпусе.

In [15]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(20)

[('и', 0.030066580918921042),
 ('в', 0.02628001641794979),
 ('не', 0.02506911192985684),
 ('на', 0.015955320798239428),
 ('что', 0.014345802260534357),
 ('я', 0.011691602907246653),
 ('а', 0.011463516055646737),
 ('с', 0.011339789697467536),
 ('это', 0.009536074571489897),
 ('ты', 0.008321404498582796),
 ('как', 0.007882444897390503),
 ('у', 0.006848522895562581),
 ('но', 0.005786090037284669),
 ('так', 0.005383172462170666),
 ('по', 0.005060945990217011),
 ('то', 0.005049649235774562),
 ('все', 0.0046537248896011225),
 ('за', 0.004583792600195488),
 ('же', 0.004228751746289958),
 ('если', 0.004209385881531474)]

In [16]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205),
 ('к', 0.00407161959610543),
 ('за', 0.0040125143695431435),
 ('россии', 0.0036751497055696383),
 ('для', 0.003325831175549828),
 ('его', 0.003260084912295149),
 ('он', 0.0031704309169478593),
 ('от', 0.003066830744546547),
 ('сообщает', 0.003050228152815567),
 ('а', 0.0029180715226369697),
 ('также', 0.002716184007188258)]

Эти вероятности уже можно использовать, чтобы ответить на вопрос - это предложение больше подходит для новостей или для анонимного форума?

В теории вероятностей для того, чтобы найти общую вероятность нескольких независимых событий произойти одновременно, нужно перемножить вероятности отдельных событий. В нашем случае мы хотим найти вероятность получить данное предложение. Для этого мы перемножаем вероятности слов в этом предложении. 

(Если бы мы сложили вероятности, то мы бы получили вероятность выбрать из корпуса 1 из слов в данном предложении)

В прошлом семинаре мы говорили о том, что вместо умножения вероятностей можно складывать логарифмы от них. Еще нам нужно учесть одну деталь - некоторых слов может не быть в словаре и, соответственно, вероятность будет нулевая. Можно использовать в таких случаях небольшое значение вероятности, например 1/длина корпуса. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов.

In [57]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    if word in probas_dvach:
        prob['dvach'] += (np.log(probas_dvach[word]))
    else:
        prob['dvach'] += (np.log(1/len(norm_dvach)))
    
    if word in probas_news:
        prob['news'] += (np.log(probas_news[word]))
    else:
        prob['news'] += (np.log(1/len(norm_news)))


Мы можем посчитать экспоненту, чтобу получить значение вероятностей, но можно сравнивать и сумму логарифмов

In [63]:
np.exp(prob['dvach']), np.exp(prob['news']) # вероятность по новостному корпусу выше

(3.8958314050721132e-50, 4.573351371331133e-45)

In [64]:
prob['dvach'], prob['news']

(-113.76934753871046, -102.09608290694159)

In [66]:
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    if word in probas_dvach:
        prob['dvach'] += (np.log(probas_dvach[word]))
    else:
        prob['dvach'] += (np.log(1/len(norm_dvach)))
    
    if word in probas_news:
        prob['news'] += (np.log(probas_news[word]))
    else:
        prob['news'] += (np.log(1/len(norm_news)))



In [67]:
np.exp(prob['dvach']), np.exp(prob['news']) # вероятность по 2ch корпусу выше

(1.0203198649400299e-34, 1.2031573431840098e-40)

In [68]:
prob['dvach'], prob['news']

(-78.26777699058295, -91.9184544989827)

Однако такая оценка вероятности предложения черезчур упрощает действительность. Слова в предложении - это не независимые события, выбор первого слова сильно влияет на вероятности выбрать второе, третье и так далее.

Такие события можно оценивать по формуле полной вероятности:

In [49]:
Image(url="https://i.ibb.co/sC7CKzQ/image.png",
     width=500, height=500)

А если простыми словами, то для того, чтобы получить вероятность предложения, нужно перемножить вероятность первого слова, вероятность второго слова, при условии первого, вероятность третьего при условии первого и второго, вероятность четвертого слова, при условии первого, второго и третьего и так далее до вероятности последнего слова при условии всех предшешевствующих.

Условные вероятности для слов можно также вычислить по частотностям. Вероятность слова А при условии слова Б равна отношению количества раз, которое встретились слова А и Б вместе, к количеству раз, которое встретилось слово Б. Вероятность слова В при условии А и Б равна отношению количества раз, которое встретились слова А,Б и В вместе к количеству раз, которое встретились слова А и Б.
И так далее. 

Но тут появляется проблема. Для того, чтобы расчитать полную вероятность предложения нужно, чтобы такое предложение уже встретилось в корпусе хотя бы 1 раз. Очевидно, что даже огромный корпус всего написанного текста не включает в себя все возможные тексты (тем более маленький корпус). Поэтому один из множителей в произведении будет нулевым, а значит и все произведение станет нулевым.

Для того, чтобы этого избежать можно поубавить строгости и предположить, что вероятность слова зависит только от предыдущего слова. Это предположение называется марковским (в честь математика Андрея Маркова). Такую модель еще можно назвать биграммной.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма. А частоты отдельных слов у нас уже есть. 

In [69]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Вероятность первого слово можно по идее считать просто как вероятность униграмма, но можно сделать небольшое добавление в нашу модель - поставить в начала каждого предложения технический токен начала предложения, а вероятность первого слова рассчитывать как вероятность биграма старт-первое слово поделить на частоту старта. Дальше мы будем генерировать текст с помощью языковой модели и это поможет нам генерировать более красивые предложения.


Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [74]:
sentences_dvach = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [75]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))


unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))


In [76]:
bigrams_news.most_common(10)

[('<start> в', 7972),
 ('<start> по', 6211),
 ('<start> как', 3738),
 ('риа новости', 3504),
 ('по словам', 1971),
 ('об этом', 1795),
 ('<start> однако', 1694),
 ('<start> на', 1643),
 ('что в', 1624),
 ('<start> об', 1619)]

Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова.

In [81]:
# phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала''Ныть надо меньше и работать больше.'
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'
prob = Counter()
for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
    word1, word2 = ngram.split()
    
    if word1 in unigrams_dvach and ngram in bigrams_dvach:
        prob['dvach'] += np.log(bigrams_dvach[ngram]/unigrams_dvach[word1])
    else:
        prob['dvach'] += np.log(2e-7)
    
    if word1 in unigrams_news and ngram in bigrams_news:
        prob['news'] += np.log(bigrams_news[ngram]/unigrams_news[word1])
    else:
        prob['news'] += np.log(2e-7)

In [82]:
prob.most_common()

[('dvach', -52.790575218724534), ('news', -125.19134723241505)]

### Генерация текста

Попробуем теперь генерироват текст с помощью биграмной языковой модели. Принцип генерации очень простой - на каждом шаге мы случайно выбираем следующее слово согласно вероятностям, расчитанным по 1 предыдущему слову.

Можно заранее расчитать все вероятности и сохранить их в матрицу. Размерность матрицы слова х слова. В каждой ячееке будет лежать вероятность получить слово б, после слова а. Слово а будет в строке, а б в колонке.

In [27]:
matrix_dvach = np.zeros((len(unigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}


for ngram in bigrams_dvach:
    word1, word2 = ngram.split()
    matrix_dvach[word2id_dvach[word1]][word2id_dvach[word2]] =  (bigrams_dvach[ngram]/
                                                                     unigrams_dvach[word1])



In [28]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_news = np.zeros((len(unigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


# вероятность расчитываем точно также
for ngram in bigrams_news:
    word1, word2 = ngram.split()
    matrix_news[word2id_news[word1]][word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])



Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [29]:

def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [30]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

в дс 2 
 я верю в этом плане способов типа не та пикча отвалилась 
 я быстро худею точно таком духе 
 а не обязана сказать к таковой а что резервы бросали тень чернобыля 10 
 аноны а не то слюнявя колбаску анусом торговать только ради самого лучшего качества 
 аутентичненько 
 никаких братушек это самое где-то вообще чем-то воняет с накатом 
 так хочется 
 я писал но потенциал блядь 
 под запрет на ебанько неизвестное говно уже был 
 проигрываю блять а там было перспективной площадкой но не точно стоит копейки на penny for you red это


In [31]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

в стране последуют в заплетах 
 церемония открытия в ближайшие часы в селении чернокозово чтобы поддержать кандидата альтернативного нынешнему кабульскому режиму 
 это сообщение о готовности акты федеральных сил в начале 2000 года на те районы афганистана утром в 240 специалистов в грузии давид тевзадзе уполномоченный собра отвечали грубостью угрожали и города ухань начинается 1 
 нато заявил кириличев в четверг они знакомились с момента коль готов ли кто-либо из каждых выборов спикер госдумы геннадий кулик считает свое согласие на рекомендациях принятых мерах рф в конце марта-начале апреля 2001 года 
 по свидетельству одного президента рф виктора черепкова 
 об


### Перплексия

Генерировать текст полностью требуется достаточно редко. И к тому же с этим более менее адекватно справляются только самые современные огромные модели. Гораздно более практичное применения языковой модели - выбрать наиболее вероятное продолжение уже введенной фразы. Вы все сталкивались с этим в своих телефонах. Если попытаться сгенировать текст только на основе предложенных слов, то получится не сильно лучше чем в текстах выше. Также языковую модель можно использовать, чтобы выбрать наиболее подходящее по контексту исправление опечатки и в этом случае совсем не важно, насколько красивые тексты она генерирует.

Но как тогда оценивать качество языковой модели? Для этого стандартно используется перплексия (на русский обычно не переводят). У перплексии есть теоретическое обоснование в теории информации и даже какая-то интерпретация, но они достаточо сложные и непонятные. На практике можно просто считать, что перплексия показывает насколько хорошо языковая модель предсказывает корпус. Чем она ниже, тем лучше.

Считается перплексия по вот такой формуле:


In [50]:
Image(url="https://i.ibb.co/Ph3sNMp/image.png",
     width=500, height=500)

Простыми словами - нам нужно расчитать вероятность текста (мы это уже научились делать выше) и возвести ее в степень (-1/N), где N это количество слов в тексте.

In [88]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

Давайте посчитаем перплексию на вероятностях отдельных слов

In [84]:
# phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'

prob = {'news':[], 'dvach':[]}

for word in normalize(phrase):
    if word in probas_dvach:
        prob['dvach'].append(np.log(probas_dvach[word]))
    else:
        prob['dvach'].append(np.log(1/len(norm_dvach)))
    
    if word in probas_news:
        prob['news'].append(np.log(probas_news[word]))
    else:
        prob['news'].append(np.log(1/len(norm_news)))

In [89]:
perplexity(prob['news'])

97714.65359201374

In [90]:
perplexity(prob['dvach'])

17738.135055632363

И сравним ее с перплексией нашей языковой модели

In [96]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала''Ныть надо меньше и работать больше.'
# phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'
prob = {'dvach':[], 'news':[]}

for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
    word1, word2 = ngram.split()
    
    if word1 in unigrams_dvach and ngram in bigrams_dvach:
        prob['dvach'].append(np.log(bigrams_dvach[ngram]/unigrams_dvach[word1]))
    else:
        prob['dvach'].append(np.log(0.00001))
    
    if word1 in unigrams_news and ngram in bigrams_news:
        prob['news'].append(np.log(bigrams_news[ngram]/unigrams_news[word1]))
    else:
        prob['news'].append(np.log(0.00001))


In [97]:
perplexity(prob['news'])

931.5697598228617

In [98]:
perplexity(prob['dvach'])

13675.075046200804

Видно, что перплексия языковой модели сильно ниже. 